In [1]:
import gradio as gr
import os
import cv2
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import io
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
import time

ENDPOINT = "https://team10eighticustomvision-prediction.cognitiveservices.azure.com/"
PREDICTION_KEY = "9FRZbiwBubFIcSZ1k88tCTCskOAZwMMAMvnFLuVJ26tlU0V0fsqjJQQJ99ALACYeBjFXJ3w3AAAIACOGOj1S"
PROJECT_ID = "4218ecac-688a-422b-9e14-2726b938f67c"
PUBLISHED_NAME = "Iteration8"
 
# Initialize the prediction client
credentials = ApiKeyCredentials(in_headers={"Prediction-key": PREDICTION_KEY})
predictor = CustomVisionPredictionClient(endpoint=ENDPOINT, credentials=credentials)

first_char = "Siot"
first_char_succeed = False
first_char_is_inprogress = False


def draw_boxes(image, predictions):
    """가장 확률이 높은 객체만 경계 상자를 그리도록 수정"""
    img = image.copy()
    
    # 예측 결과 중 확률이 가장 높은 하나를 선택
    if predictions:
        highest_prediction = max(predictions, key=lambda p: p.probability)
        
        if first_char_is_inprogress:
            print(highest_prediction.tag_name, highest_prediction.probability)
            if highest_prediction.tag_name == first_char and highest_prediction.probability > 0.7:
                global first_char_succeed
                first_char_succeed = True
                print("First character detected successfully!")
        
        # 확률이 0.5 이상인 객체만 선택
        if highest_prediction.probability > 0.5:
            color = (255, 0, 0)  # 경계 상자 색상 (빨간색)
            box = highest_prediction.bounding_box
            left = int(box.left * img.shape[1])
            top = int(box.top * img.shape[0])
            width = int(box.width * img.shape[1])
            height = int(box.height * img.shape[0])

            # 경계 상자 그리기
            cv2.rectangle(img, 
                        (left, top), 
                        (left + width, top + height), 
                        color, 
                        2)
            
            # 라벨과 확률 텍스트 추가
            label = f"{highest_prediction.tag_name}: {highest_prediction.probability:.2f}"
            cv2.putText(img, 
                        label, 
                        (left, top - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, 
                        color, 
                        2)
    
    return img


def process_frame(frame):
    if frame is None:
        return None
    
    # Convert frame to PIL Image
    pil_image = Image.fromarray(frame)
    
    # Save to bytes for Azure API
    img_byte_arr = io.BytesIO()
    pil_image.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()

    try:
        # Get predictions from Azure Custom Vision
        results = predictor.detect_image(PROJECT_ID, PUBLISHED_NAME, img_byte_arr)
        
        # Draw boxes on frame
        annotated_frame = draw_boxes(frame, results.predictions)
        
        return annotated_frame
    
    except Exception as e:
        print(f"Error during prediction: {e}")
        return frame

def check_first_success():
    global first_char_succeed, first_char_is_inprogress
    
    first_char_is_inprogress = True
    
    time.sleep(5)
    
    first_char_is_inprogress = False
    
    if first_char_succeed:
        return gr.Image(value="./images/Correct/Siot.jpg", label="Input Image", type="pil")
    
    return gr.Image(value="./images/Basic/Siot.jpg", label="Input Image", type="pil")

# Blocks section
with gr.Blocks() as demo:
    with gr.Row():  # Row layout for input and output
        with gr.Column():  # Input webcam column
            first_image = gr.Image(value="./images/Basic/Siot.jpg", label="Input Image", type="pil")
            detect_button = gr.Button("Detect Objects")
            detect_button.click(fn=check_first_success, inputs=None, outputs=first_image)
            
    with gr.Row():  # Row layout for input and output
        with gr.Column():  # Input webcam column
            webcam_input = gr.Image(sources="webcam", streaming=True, mirror_webcam=True, label="Webcam")
        with gr.Column():  # Output display column
            output = gr.Image(label="Detected Objects")

    # Set the process_frame function as the update function for webcam input
    webcam_input.stream(process_frame, inputs=webcam_input, outputs=output)
    
    demo.title = "Azure Custom Vision Object Detection"
    demo.description = "Real-time object detection using Azure Custom Vision"

def init_global_vars():
    global first_char_succeed, first_char_is_inprogress
    first_char_succeed = False
    first_char_is_inprogress = False

if __name__ == "__main__":
    init_global_vars()
    demo.launch(share=True)


c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://75bcfbd488acfddfad.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Yo 0.642857
Yu 0.40827152
Yo 0.4656968
Yo 0.5872162
Yo 0.7118783
Yo 0.7973189
Yo 0.7379643
Yo 0.67788845
Yu 0.6109421
Yu 0.57305133
Yu 0.62193817
Yu 0.6545785
Yu 0.6739745
Yu 0.5363767
Yu 0.70355755
Yo 0.3872199
Yae 0.32673338
Nieun 0.26024154
Rieul 0.8785918
Rieul 0.8664876
Yo 0.7336812
O 0.7035176
Yo 0.43824908
Yo 0.6237183
Yae 0.95525837
Siot 0.9178469
First character detected successfully!
Siot 0.99482214
First character detected successfully!
Siot 0.99483216
First character detected successfully!
